In [1]:
import collections

import pandas as pd

In [2]:
%cd ../..

/Users/mwtmurphy/projects/advent-of-code


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/advent-of-code-rjxGAg1e-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
map_7a = {"T": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
map_7b = {"T": 10, "J": 1, "Q": 12, "K": 13, "A": 14}

def map_7a_card(card: str, char_map: dict = map_7a) -> int:
    '''Returns integer value of card'''
    
    if card in ["T", "J", "Q", "K", "A"]:
        card_val = char_map[card]
    else:
        card_val = int(card)

    return card_val

def map_7b_card(card: str, char_map: dict = map_7b) -> int:
    '''Returns integer value of card'''
    
    if card in ["T", "J", "Q", "K", "A"]:
        card_val = char_map[card]
    else:
        card_val = int(card)

    return card_val

def rank_7b_hand(hand: pd.Series) -> int:
    hand_counter = collections.Counter(hand)

    if len(hand_counter) == 1:
        return 7 # five of a kind
    
    if "J" in hand_counter:
        n_jacks = hand_counter.pop("J")
        card_counts = [card[1] for card in hand_counter.most_common()]

        if n_jacks == 4:
            return 7 # high card -> five of a kind
        
        elif n_jacks == 3:
            if card_counts[0] == 2:
                return 7 # one pair -> five of a kind
            else:
                return 6 # high card -> four of a kind
        
        elif n_jacks == 2:
            if card_counts[0] == 3:
                return 7 # three of a kind -> five of a kind
            elif card_counts[0] == 2:
                return 6 # one pair -> four of a kind
            else:
                return 4 # high card -> three of a kind
        
        else:
            if card_counts[0] == 4:
                return 7 # four of a kind -> five of a kind
            elif card_counts[0] == 3:
                return 6 # three of a kind -> four of a kind
            elif card_counts[0] == 2:
                if card_counts[1] == 2:
                    return 5 # two pair -> full house
                else:
                    return 4 # one pair -> three of a kind
            else:
                return 2 # high card -> one pair


    card_counts = [card[1] for card in hand_counter.most_common()]

    if len(hand_counter) == 2:
        if card_counts[0] == 4:
            return 6 # four of a kind
        else:
            return 5 # full house
    
    elif len(hand_counter) == 3:
        if card_counts[0] == 3:
            return 4 # three of a kind
        else:
            return 3 # two pair
    
    elif len(hand_counter) == 4:
        return 2 # one pair
    
    else:
        return 1 # high card

In [4]:
with open("data/2023/day_7.txt") as file:
    day_7_lines = file.read().strip().splitlines()


In [5]:
hands = [line.split(" ")[0] for line in day_7_lines]
hand_df = pd.DataFrame([[map_7a_card(card) for card in hand] for hand in hands])

type_rank = []

# group index values by hand type
for ix, hand in hand_df.iterrows():
    hand_counter = collections.Counter(hand)

    if len(hand_counter) == 1:
        type_rank.append(7) # five of a kind

    elif len(hand_counter) == 2:
        if hand_counter.most_common(1)[0][1] == 4:
            type_rank.append(6) # four of a kind
        else:
            type_rank.append(5) # full house
    
    elif len(hand_counter) == 3:
        if hand_counter.most_common(1)[0][1] == 3:
            type_rank.append(4) # three of a kind
        else:
            type_rank.append(3) # two pair
    
    elif len(hand_counter) == 4:
        type_rank.append(2) # one pair
    
    else:
        type_rank.append(1) # high card

hand_df["type"] = type_rank

hand_df["bid"] = [line.split(" ")[1] for line in day_7_lines]
hand_df["bid"] = hand_df["bid"].astype(int)

hand_df = hand_df.sort_values(by=["type", 0, 1, 2, 3, 4], ascending=False).reset_index(drop=True)
hand_df["winnings"] = (hand_df.shape[0] - hand_df.index) * hand_df["bid"]

total_winnings = hand_df["winnings"].sum()
print("day 7a solution:", total_winnings)

day 7a solution: 255048101


In [6]:
hands = [line.split(" ")[0] for line in day_7_lines]
hand_df = pd.DataFrame([[card for card in hand] for hand in hands])

# group index values by hand type
type_rank = []
for _, hand in hand_df.iterrows():
    type_rank.append(rank_7b_hand(hand))

# append mapped hand info for 7b to hand df
mapped_hand_df = pd.DataFrame([[map_7b_card(card) for card in hand] for hand in hands])
mapped_hand_df.columns = [f"map_{i}" for i in range(5)]
hand_df = pd.concat([hand_df, mapped_hand_df], axis=1)

# calculate winnings
hand_df["type"] = type_rank
hand_df["bid"] = [line.split(" ")[1] for line in day_7_lines]
hand_df["bid"] = hand_df["bid"].astype(int)

sort_cols = ["type", "map_0", "map_1", "map_2", "map_3", "map_4"]
hand_df = hand_df.sort_values(by=sort_cols, ascending=False).reset_index(drop=True)
hand_df["winnings"] = (hand_df.shape[0] - hand_df.index) * hand_df["bid"]

total_winnings = hand_df["winnings"].sum()
print("day 7b solution:", total_winnings)

day 7b solution: 253718286
